In [42]:
import sys
# sys.path.append('/mnt/6088FEB688FE89B4/Users/arsen/Desktop/code/git/python-utils-ak')
sys.path.append(r'C:\Users\arsen\Desktop\code\git\python-utils-ak')
from utils_ak.reflexion import load as _load

def _import_globals(module):
    for x in dir(module):
        if not x.startswith('_'):
            globals()[x] = getattr(module, x)
            
def load(module_obj, module_name=None, reload=False, import_globals=False):
    module = _load(module_obj, module_name, reload, import_globals=False)
    if import_globals:
        _import_globals(module)
    return module

# Models

In [43]:
request = {'Моцарелла Фиор ди Латте в воде "Красная птица", 45%, 0,125 кг, ф/п': 1, 
          'Моцарелла Фиор Ди Латте в воде "Pretto", 45%, 0,125 кг, ф/п, (8 шт)': 1}

In [44]:
import pandas as pd

In [45]:
df = pd.DataFrame(request.items(), columns=['name', 'qty'])
df

,name,qty
0,"Моцарелла Фиор ди Латте в воде ""Красная птица""...",1
1,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, ...",1


In [46]:
import sys
# sys.path.append('/mnt/6088FEB688FE89B4/Users/arsen/Desktop/code/git/2020.10-umalat/umalat')
sys.path.append(r'C:\Users\arsen\Desktop\code\git\2020.10-umalat\umalat')


In [47]:
from app.models import *
from app.models import Boiling as BoilingModel

In [48]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# sqlite_filepath = '/home/akadaner/Desktop/data.sqlite'
sqlite_filepath = r"C:\Users\arsen\Desktop\code\git\2020.10-umalat\umalat\data.sqlite"
engine = create_engine(f"sqlite:///{sqlite_filepath}")
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [49]:
df['sku'] = df['name'].apply(lambda s: session.query(SKU).filter(SKU.name == s).first())

In [50]:
df

,name,qty,sku
0,"Моцарелла Фиор ди Латте в воде ""Красная птица""...",1,<SKU 27>
1,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, ...",1,<SKU 24>


In [51]:
sku = df.iloc[0]['sku']

In [52]:
session.query(BoilingModel).first()

<Boiling 1>

In [53]:
df['boiling'] = df['sku'].apply(lambda sku: session.query(BoilingModel).filter(BoilingModel.id == sku.boiling_id).first())

In [54]:
df

,name,qty,sku,boiling
0,"Моцарелла Фиор ди Латте в воде ""Красная птица""...",1,<SKU 27>,<Boiling 6>
1,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, ...",1,<SKU 24>,<Boiling 6>


In [55]:
lines = ['cheese_maker', 'melting', 'termizator', 'packing']

# Time

In [56]:
def cast_t(obj):
    if isinstance(obj, int):
        return obj
    elif isinstance(obj, str):
        # 12:00
        hours, minutes = obj.split(':')
        return int(hours) * 12 + int(int(minutes) / 5)
    else:
        raise Exception('Unknown format')
        
def cast_time(obj):
    if isinstance(obj, str):
        return obj
    elif isinstance(obj, int):
        hours = obj // 12
        hours = hours % 24
        minutes = obj % 12 * 5
        return f'{hours:02}:{minutes:02}'

# Block

In [57]:
module = load('2020.11.18 algo.py', import_globals=True, reload=True)

In [102]:
root = Block('root', t=cast_t('01:00'), index_width=4)

with root.make('boiling', block_num=12) as boiling:
    with boiling.make('water_pouring', method='add', t=cast_t('01:45')) as water_pouring:
        total_size = 27

        with water_pouring.make(y=0) as b:
            b.make('pouring_label', size=6, block_num=12)
            b.make('pouring_name', size=total_size - 6, name='чильеджина  3,6 альче  8500кг')
        with water_pouring.make(y=1) as b:
            b.make('pouring_and_fermenting', size=8)
            b.make('soldification', size=7)
            b.make('cutting', size=7)
            b.make('pouring_off', size=3)
            b.make('extra', size=total_size - 8 - 7 - 7 - 3)

    boiling.make('termizator', size=6, visible=False)

    with boiling.make('melting', method='add', t=cast_t('08:00')) as melting:
        total_size = 25
        with melting.make(y=0) as b:
            b.make('melting_label', size=3)
            b.make('melting_name', size=total_size - 3, name='чильеджина 5кг/сердечки/фдл 0,125(безлактозная)/0,1')
        with melting.make(y=1) as b:
            b.make('serving', size=6)
            b.make('melting_process', size=total_size - 6, speed=900)
        with melting.make(y=2) as b:
            b.make(size=6, visible=False)
            b.make('cooling1', size=5)
            b.make('cooling2', size=10)
    
    with boiling.make('packing', method='add', t=cast_t('08:30')) as packing:
        total_size = 25
        with packing.make(y=0) as b:
            b.make('packing_label', size=3) 
            b.make('packing_name', size=total_size - 3, name='чильеджина 5кг/сердечки/фдл 0,125(безлактозная)/0,1')
        with packing.make(y=1) as b:
            b.make('packing_brand', size=total_size, name='безлактозная/претто')
        with packing.make(y=2) as b:
            b.make('configuration', size=3, visible=False)
            b.make('configuration', size=1)
            b.make('configuration', size=5, visible=False)
            b.make('configuration', size=1)

In [163]:
# ROWS = {'water_1': 1, 'water_2': 3, 'salt_1': 5, 'salt_2': 7, 'water_melting': 9, 'water_packing': 12, 'salt_melting': 15, 'salt_packing': 30}
ROWS = {'water_1': 1, 'water_2': 3, 'water_melting': 5, 'water_packing': 8}

def cast_row(obj):
    if isinstance(obj, int):
        return obj
    elif isinstance(obj, str):
        return ROWS[obj]
    else:
        raise Exception('Unknown object type')

In [158]:
style = {
    'water_pouring': {'beg_time': '01:00', 'y': cast_row('water_1')},
    'pouring_label': {'text': '{block_num} налив'},
    'pouring_name': {'text': '{name}'},
    'pouring_and_fermenting': {'text': 'налив/внесение закваски', 'color': 'orange'},
    'soldification': {'text': 'схватка', 'color': 'yellow'},
    'cutting': {'text': 'резка/обсушка', 'color': '#92d050'},
    'pouring_off': {'text': 'слив', 'color': 'red'},
    'melting': {'beg_time': '07:00', 'y': cast_row('water_melting')},
    'melting_label': {'text': '{block_num}'},
    'melting_name': {'text': '{name}'},
    'serving': {'color': 'orange', 'text': 'подача и вымешивание'},
    'melting_process': {'color': 'orange', 'text': 'плавление {speed} кг/ч'},
    'cooling1': {'text': 'охлаж1 <{size} * 5> мин'},
    'cooling2': {'text': 'охлаж2 <{size} * 5> мин'},
    'packing': {'beg_time': '07:00', 'y': cast_row('water_packing')},
    'packing_label': {'text': '{block_num}'},
    'packing_name': {'text': '{name}'},
    'packing_brand': {'color': '#f2dcdb', 'text': '{name}'},
    'configuration': {'color': 'red'}
}

In [ ]:
# update styles
for b in root:
    if b.block_class in style:
        b.local_props.update(style[b.block_class])

# Drawing

In [161]:
from matplotlib import colors

def cast_color(obj):
    if isinstance(obj, (tuple, list)):
        if all(isinstance(x, int) for x in obj):
            return '#{:02x}{:02x}{:02x}'.format(*obj)
        elif all(isinstance(x, float) for x in obj):
            return colors.to_hex(obj)
    elif isinstance(obj, str):
        try:
            return cast_color(colors.to_rgb(obj))
        except:
            pass
    raise Exception('Unknown object type')

In [146]:
import pandas as pd
from openpyxl import Workbook
from datetime import datetime
import openpyxl as opx
from openpyxl.styles import Alignment
from openpyxl.styles import PatternFill
import json

def draw_block(x, y, w, h, text, colour):
    if not colour:
        colour = cast_color('white') # default white colour
    SHEET.merge_cells(start_row=y, start_column=x, end_row=y + h - 1, end_column=x + w - 1)
    merged_cell = SHEET.cell(row=y, column=x)  
    merged_cell.value = text
    merged_cell.alignment = Alignment(horizontal='center')
    merged_cell.fill = PatternFill("solid", fgColor=colour[1:])

In [147]:
def draw(block):
    if block.children:
        for child in block.children:
            draw(child)
    else:
        props = block.props or {}
        props.update(style.get(block.block_class, {}))
#         print(block.block_class, props)
        text = props.get('text', '')
        color = cast_color(props.get('color', 'white'))
        
        if block.props.get('visible') == False:
            return
        
        text = text.format(**props)
        text = text.replace('<', '{')
        text = text.replace('>', '}')
        text = eval(f'f{text!r}')
        
        beg = block.beg
        beg -= cast_t(block.props['beg_time']) # shift of timeline
        beg += block.props['index_width'] # first index columns
        beg += 1 # indexing starts with 1 in excel
        return draw_block(beg, block.props['y'], block.props['size'], 1, text, color)

In [148]:
def move_sheet(wb, from_loc=None, to_loc=None):
    sheets=wb._sheets

    # if no from_loc given, assume last sheet
    if from_loc is None:
        from_loc = len(sheets) - 1

    #if no to_loc given, assume first
    if to_loc is None:
        to_loc = 0

    sheet = sheets.pop(from_loc)
    sheets.insert(to_loc, sheet)
    
# work_book = opx.load_workbook(r'2020.11.18 schedule_template.xlsx')

# # delete all but last - original
# for sheet in work_book.worksheets[:-1]:
#     work_book.remove_sheet(sheet)
# # copy sheet to work with
# SHEET = work_book.copy_worksheet(work_book.worksheets[0])
# # put it at the beginning
# move_sheet(work_book, 1, 0)

In [154]:
from openpyxl.utils import get_column_letter
work_book = opx.Workbook()
SHEET = work_book.worksheets[0]
for i in range(288):
    SHEET.column_dimensions[get_column_letter(i+1)].width = 1.2

In [155]:
draw(root)

In [156]:
work_book.save('output.xlsx')